In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db


'Connected: None@chinook.db'

This project uses the Chinook database, which contains information about a music shop. 

The commands preceded by % are built-in magic commands that allow for things like output to be displayed in Jupyter notebooks.

The command %%sql tells the Jupyter notebook "Get ready for some SQL." 

The command "%sql sqlite:///chinook.db" allows us to connect to this database. Let's explore what's inside it using SQL's  "type" attribute.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


The query above shows that the Chinook database contains eleven tables. 

Since I'm connected to the Chinook database, Jupyter will assume that any of my FROM statements refer to these tables.

The tables themselves contain columns. We'll take a look at a few of them below- specifically invoice line, track, and genre.

In [3]:
%%sql
SELECT * FROM invoice_line
LIMIT 5;

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


In [4]:
%%sql
SELECT * FROM track
LIMIT 5;

Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99
5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


In [5]:
%%sql
SELECT * FROM genre;

Done.


genre_id,name
1,Rock
2,Jazz
3,Metal
4,Alternative & Punk
5,Rock And Roll
6,Blues
7,Latin
8,Reggae
9,Pop
10,Soundtrack


Say we want to select three tracks of different genres to add to our store.

One way to determine which tracks would sell best would be to discover which three music genres sell best in the USA. We can do this by checking the number of tracks sold by Genre in the USA as a percentage of all tracks sold in the USA.

First, let's get the denominator of that percentage by finding the number of tracks purchased by customers with addresses in the USA.

In [6]:
%%sql
SELECT c.first_name,
    c.state,
    SUM(il.quantity) total_quantity_usa
FROM customer c
LEFT JOIN invoice i ON i.customer_id=c.customer_id
LEFT JOIN invoice_line il ON il.invoice_id=i.invoice_id
WHERE c.country="USA";

Done.


first_name,state,total_quantity_usa
Julia,UT,1051


We can use this query as a sub-query in our final query.

In [7]:
%%sql
SELECT i.billing_country country,
    g.name genre_name,
    SUM(il.quantity) genre_quantity_USA,
    CAST(SUM(il.quantity) AS FLOAT) / (SELECT SUM(il.quantity) total_quantity_usa
                                        FROM customer c
                                        LEFT JOIN invoice i ON i.customer_id=c.customer_id
                                        LEFT JOIN invoice_line il ON il.invoice_id=i.invoice_id
                                        WHERE c.country="USA") percentage_tracks_USA
FROM track t
INNER JOIN genre g ON g.genre_id=t.genre_id
INNER JOIN invoice_line il ON il.track_id=t.track_id
INNER JOIN invoice i ON i.invoice_id=il.invoice_id
WHERE country="USA"
GROUP BY country, genre_name
ORDER BY genre_quantity_USA DESC;

Done.


country,genre_name,genre_quantity_USA,percentage_tracks_USA
USA,Rock,561,0.5337773549000951
USA,Alternative & Punk,130,0.12369172216936251
USA,Metal,124,0.11798287345385347
USA,R&B/Soul,53,0.05042816365366318
USA,Blues,36,0.03425309229305423
USA,Alternative,35,0.03330161750713606
USA,Latin,22,0.02093244529019981
USA,Pop,22,0.02093244529019981
USA,Hip Hop/Rap,20,0.019029495718363463
USA,Jazz,14,0.013320647002854425


In the USA, Punk, Blues, and Pop are ahead of Hip-Hop, which suggests that we should add tracks from these genres to our USA catalogue.

Next, we'll analyze the performance of different sales support agents, specifically the value of purchases associated with some of the top and lowest performers.

In [8]:
%%sql
SELECT DISTINCT e.first_name || " " || e.last_name full_name,
        e.employee_id,
        e.title
FROM employee e
LEFT JOIN customer c ON c.support_rep_id=e.employee_id;

Done.


full_name,employee_id,title
Andrew Adams,1,General Manager
Nancy Edwards,2,Sales Manager
Jane Peacock,3,Sales Support Agent
Margaret Park,4,Sales Support Agent
Steve Johnson,5,Sales Support Agent
Michael Mitchell,6,IT Manager
Robert King,7,IT Staff
Laura Callahan,8,IT Staff


Note also that there are only three Sales Support Agents for a comparatively large number of customers.

This strongly suggests that we will need a left join, with employees as the left table.


In [9]:
%%sql
SELECT e.first_name || " " || e.last_name full_name,
        e.title,
        date(e.hire_date) hire_date,
        SUM(i.total) total_sales
FROM employee e
LEFT JOIN customer c ON c.support_rep_id=e.employee_id
INNER JOIN invoice i ON i.customer_id=c.customer_id
GROUP BY full_name;

Done.


full_name,title,hire_date,total_sales
Jane Peacock,Sales Support Agent,2017-04-01,1731.510000000004
Margaret Park,Sales Support Agent,2017-05-03,1584.0000000000032
Steve Johnson,Sales Support Agent,2017-10-17,1393.9200000000028


Note that Jane Peacock has the most sales, but has also been working at the Chinook record store for the longest out of all of the Sales Support Agents. 

For the next task, we need to calculate the total number of customers, total value of sales, average value of sales per customer, and the average order value.

Some countries only have one customer. We'll combine these countries into a category called "Other," but we'll need to find and flag them before we do. 

In [10]:
%%sql

SELECT
    CASE
        WHEN (SELECT COUNT(*)
             FROM customer
             WHERE country=c.country)=1 THEN "Other"
        ELSE c.country
    END 
    AS new_country,
        COUNT(*) total_customers
FROM customer c
GROUP BY new_country;

Done.


new_country,total_customers
Brazil,5
Canada,8
Czech Republic,2
France,5
Germany,4
India,2
Other,15
Portugal,2
USA,13
United Kingdom,3


How does the code above work? 

The secret is in the CASE statement, which loops through each country in the customer table, counts the data records for each country, and records the result in a column named COUNT(\*). 

Specifically, the CASE statement is:

1.) looping through the subquery's "customer" table's country column and picking a value, 
2.) checking if that value is in the country column in the FROM clause's "customer" table, 
3.) selecting only records with that value from the "customer" table, and 
4.) counting all of the records in that selection and recording the count in a column named COUNT(\*).

Since the CASE state loops through one value at a time, the subquery returns a single value. If that value is "1", the column "new_country" gets the value "Other." If not, it gets the original name of the country.

Now that we know how the subquery works, we can expand it.

In [11]:
%%sql
WITH 
customers_by_country AS
(
    SELECT
        CASE
            WHEN (SELECT COUNT(*)
                 FROM customer
                 WHERE country=c.country)=1 THEN "Other"
            ELSE c.country
        END 
        AS new_country,
        c.customer_id,
        il.*
    FROM customer c
    INNER JOIN invoice i ON i.customer_id=c.customer_id
    INNER JOIN invoice_line il ON il.invoice_id=i.invoice_id 
)
SELECT country,
        total_customers,
        total_sales_value,
        average_customer_sales,
        average_order_value
FROM
(
SELECT 
    new_country country,
    COUNT(DISTINCT customer_id) total_customers,
    SUM(unit_price) total_sales_value,
    SUM(unit_price) / COUNT(DISTINCT customer_id) average_customer_sales,
    SUM(unit_price) / COUNT(DISTINCT invoice_id) average_order_value,
    CASE
        WHEN new_country="Other" THEN 0
        ELSE 1
    END
    AS sort_index
FROM customers_by_country
GROUP BY new_country
ORDER BY sort_index DESC, total_sales_value DESC
    )
;

Done.


country,total_customers,total_sales_value,average_customer_sales,average_order_value
USA,13,1040.490000000008,80.03769230769292,7.942671755725252
Canada,8,535.5900000000034,66.94875000000043,7.047236842105309
Brazil,5,427.68000000000245,85.53600000000048,7.011147540983647
France,5,389.0700000000021,77.81400000000042,7.781400000000042
Germany,4,334.6200000000016,83.6550000000004,8.161463414634186
Czech Republic,2,273.24000000000103,136.62000000000052,9.108000000000034
United Kingdom,3,245.52000000000078,81.84000000000026,8.768571428571457
Portugal,2,185.13000000000022,92.56500000000011,6.3837931034482835
India,2,183.1500000000002,91.5750000000001,8.72142857142858
Other,15,1094.9400000000085,72.99600000000056,7.448571428571486


The last question we need to answer is: what percentage of purchases are individual tracks, and what percentage are whole albums. 

This is a little tricky, but the answer is important: the record store might not need to buy entire albums itself if demand for its product is limited to tracks. Note that the cost of an album is the cost of its separate tracks.

The simplest way to do this is to determine if an invoice_id has all tracks from an album. If so, we'll consider that invoice to be an album purchase. If not, we'll consider the purchases to be individual.

First, we'll make a selection that tells us how many tracks were purchased from each album for each invoice.

In [47]:
%%sql
SELECT il.invoice_id,
    t.album_id,
    t.track_id,
    a.title album_title,
    COUNT(il.quantity) number_of_tracks
FROM track t
INNER JOIN invoice_line il ON il.track_id=t.track_id
INNER JOIN album a ON a.album_id=t.album_id
GROUP BY il.invoice_id, t.album_id
LIMIT 400;

Done.


invoice_id,album_id,track_id,album_title,number_of_tracks
1,91,1173,Use Your Illusion I,16
2,20,201,The Best Of Buddy Guy - The Millenium Collection,1
2,34,392,Chill: Brazil (Disc 2),1
2,39,482,International Superhits,1
2,66,819,The Battle Rages On,1
2,73,1119,Unplugged,1
2,134,1641,Led Zeppelin III,1
2,190,2324,The Best Of R.E.M.: The IRS Years,1
2,215,2651,The Police Greatest Hits,1
2,218,2701,Voodoo Lounge,1


Note that there are some invoices where people buy a number of single tracks from multiple albums, and others where people buy multiple tracks from a single album.

Next, we'll get an idea of how many tracks there are in each album.

In [24]:
%%sql
SELECT a.album_id,
    a.title,
    count(t.track_id) number_of_tracks
FROM album a
LEFT JOIN track t ON t.album_id=a.album_id
WHERE a.album_id=91
GROUP BY a.album_id
ORDER BY a.album_id
;

Done.


album_id,title,number_of_tracks
91,Use Your Illusion I,16


We're primarily interested in two columns from the two selections: album ID, and count of tracks. 

But we also need to consider the the invoice ID, because we need to

1. loop through the invoice ID (so they should be ordered), </br>
2. create a selection for each invoice, </br>
3. compare the number of tracks purchased in an invoice with the tracks available for an album, </bt>
4. create a new case column that indicates if an invoice's purchases per album constituted purchasing the entire album; we assume this is the case if invoice x purchased y tracks from album z when album z has exactly y tracks.


In [14]:
%%sql
WITH 
invoice_tracks AS 
(
    SELECT il.invoice_id,
        t.album_id,
        COUNT(il.quantity) number_of_tracks_invoice
    FROM track t
    INNER JOIN invoice_line il ON il.track_id=t.track_id
    INNER JOIN album a ON a.album_id=t.album_id
    GROUP BY il.invoice_id, t.album_id
    ORDER By il.invoice_id, a.album_id
),
album_tracks AS
(
    SELECT a.album_id,
        count(t.track_id) number_of_tracks_album
    FROM album a
    LEFT JOIN track t ON t.album_id=a.album_id
    GROUP BY a.album_id
    ORDER BY a.album_id
),
combined_tracks AS
(
select it.invoice_id,
        at.album_id,
        at.number_of_tracks_album,
        it.number_of_tracks_invoice,
        CASE
            WHEN at.number_of_tracks_album=it.number_of_tracks_invoice 
            AND at.number_of_tracks_album >= 2 THEN "yes"
            ELSE "no"
            END 
        AS entire_album_purchase
from album_tracks at
LEFT JOIN invoice_tracks it ON it.album_id=at.album_id
)

SELECT  CAST ((
        SELECT COUNT(*)
        FROM combined_tracks
        WHERE entire_album_purchase="yes"
        ) AS float) / CAST(COUNT(DISTINCT invoice_id) AS float) percentage_entire_album
FROM combined_tracks
;

Done.


percentage_entire_album
0.18078175895765472


In [28]:
%%sql
WITH 
    invoice_track_reference AS
        (
            SELECT il.invoice_id,
                   MIN(il.track_id) ref_track_id
            FROM invoice_line il
            GROUP BY invoice_id
        )
        
SELECT *
FROM invoice_track_reference itr
LIMIT 50
;



Done.


invoice_id,ref_track_id
1,1158
2,201
3,2516
4,748
5,1986
6,30
7,42
8,81
9,196
10,2663


In [76]:
%%sql
WITH 
    invoice_track_reference AS
        (
            SELECT invoice_id,
                   MIN(track_id) ref_track_id
            FROM invoice_line
            GROUP BY invoice_id
        )

SELECT purchased_album,
    COUNT(invoice_id) number_invoices,
    CAST(COUNT(invoice_id) AS FLOAT)/CAST((SELECT COUNT(*)
                                          FROM INVOICE) AS FLOAT) percentage
FROM
    (
    SELECT itr.*,
        CASE
            WHEN
                (
                   SELECT t.track_id FROM track t
                   WHERE album_id = (SELECT album_id FROM track t2
                                     WHERE t2.track_id=itr.ref_track_id)
               
                   EXCEPT
                
                   SELECT track_id from invoice_line
                   WHERE invoice_id=itr.invoice_id
                ) IS NULL
            AND
                (
                   SELECT track_id from invoice_line
                   WHERE invoice_id=itr.invoice_id
                
                   EXCEPT
            
                   SELECT t.track_id FROM track t
                   WHERE album_id = (SELECT album_id FROM track t2
                                 WHERE t2.track_id=itr.ref_track_id)
                ) IS NULL
            THEN "Yes"
            ELSE "No"
        END AS purchased_album
    FROM invoice_track_reference itr
    )
GROUP BY purchased_album
;


Done.


purchased_album,number_invoices,percentage
No,500,0.8143322475570033
Yes,114,0.18566775244299674


In [17]:

Next steps:
    Find Album using CASE Statement to match track_id from itr with track id in album
    For that album Except and Inverse-Except statement for invoice tracks and album tracks 
    If both are null, then Album purchase


SyntaxError: invalid syntax (<ipython-input-17-1748af53fe6b>, line 2)

In [75]:
%%sql
SELECT *
FROM invoice;

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98
4,18,2017-01-06 00:00:00,627 Broadway,New York,NY,USA,10012-2612,7.92
5,27,2017-01-07 00:00:00,1033 N Park Ave,Tucson,AZ,USA,85719,16.83
6,31,2017-01-10 00:00:00,194A Chain Lake Drive,Halifax,NS,Canada,B3S 1C5,1.98
7,49,2017-01-12 00:00:00,Ordynacka 10,Warsaw,None,Poland,00-358,10.89
8,59,2017-01-13 00:00:00,"3,Raj Bhavan Road",Bangalore,None,India,560001,9.9
9,18,2017-01-18 00:00:00,627 Broadway,New York,NY,USA,10012-2612,8.91
10,31,2017-01-18 00:00:00,194A Chain Lake Drive,Halifax,NS,Canada,B3S 1C5,1.98
